https://dumps.wikimedia.org/jawiki/20250420/
<br>
https://dumps.wikimedia.org/jawiki/

In [ ]:
import requests
url_index = "https://dumps.wikimedia.org/jawiki/20250420/jawiki-20250420-pages-articles-multistream-index.txt.bz2"
url_xml = "https://dumps.wikimedia.org/jawiki/20250420/jawiki-20250420-pages-articles-multistream.xml.bz2"
urls = [url_index, url_xml]

<h5> Download wikidump </h5>

In [ ]:
for url in urls:
    output_file = url.split("/")[-1]
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"File downloaded successfully: {output_file}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

<h5> Extract Index file </h5>

In [ ]:
!bunzip2 jawiki-20250420-pages-articles-multistream-index.txt.bz2

<h5> Peek the contents </h5>

In [ ]:
!grep Category jawiki-20250420-pages-articles-multistream-index.txt

<h5> Extract blocks </h5>

In [ ]:
import bz2
f = open("jawiki-20250420-pages-articles-multistream.xml.bz2", "rb")
f.seek(361332488)

In [ ]:
block = f.read(361635009 - 361332488)
data = bz2.decompress(block)
xml = data.decode(encoding="utf-8")
print(len(xml))
print(xml)

In [ ]:
with open("./wikidump/wikidump.xml", "w", encoding="utf-8") as xml_file:
    xml_file.write(xml)

<h5> Extract TEXT from XML file </h5>

In [ ]:
import xml.etree.ElementTree as ET

with open("./wikidump/wikidump.xml", "rb") as xml_file:
    xml = xml_file.read().decode("utf-8")

root = ET.fromstring("<root>" + xml + "</root>")

In [ ]:
import json
with open("./wikidump/wikidump.jsonl", "w", encoding="utf-8") as jsonl_file:
    for page in root.findall("page"):
        revision = page.find("revision")
        if revision is not None:
            text = revision.find("text")
            if text is not None and text.text is not None:
                jsonl_file.write(json.dumps({"text": text.text}, ensure_ascii=False) + "\n")

print("JSONL file 'wikidump2.jsonl' has been created.")